In [1]:
# !pip install requests 
!pip install python-dotenv
# !pip install json

### 01. 패키지 불러오기

In [6]:
import requests
import pandas as pd
from dotenv import load_dotenv
import os
import json
os.chdir('/Users/jun/GitStudy/Data_4/src/challenge/week2')
# load .env
load_dotenv()

True

### 02. Requests 를 통해 웹사이트 HTML 가져오기

In [4]:
# OpenWeather API 요청을 위한 기본 설정
# (https://openweathermap.org/api) 
api_key = os.environ.get('API_KEY')  # 본인의 API 키를 입력하세요.
city_name = "Seoul" # 도시는 "서울"로 설정
url = f"http://api.openweathermap.org/data/2.5/weather?q={city_name}&units=metric&appid={api_key}"
response = requests.get(url)
print(response.status_code)

200


### 03. Python 딕셔너리를 통해 HTML 파싱 후 출력

In [5]:
# API 응답 데이터 처리
data = response.json()
print(json.dumps(data, indent=4))

{
    "coord": {
        "lon": 126.9778,
        "lat": 37.5683
    },
    "weather": [
        {
            "id": 803,
            "main": "Clouds",
            "description": "broken clouds",
            "icon": "04d"
        }
    ],
    "base": "stations",
    "main": {
        "temp": 21.66,
        "feels_like": 21.21,
        "temp_min": 18.69,
        "temp_max": 21.66,
        "pressure": 1018,
        "humidity": 51,
        "sea_level": 1018,
        "grnd_level": 1012
    },
    "visibility": 10000,
    "wind": {
        "speed": 2.24,
        "deg": 102,
        "gust": 3.48
    },
    "clouds": {
        "all": 76
    },
    "dt": 1731650948,
    "sys": {
        "type": 1,
        "id": 5509,
        "country": "KR",
        "sunrise": 1731622314,
        "sunset": 1731658901
    },
    "timezone": 32400,
    "id": 1835848,
    "name": "Seoul",
    "cod": 200
}


In [32]:
# 테이블화 
pd.json_normalize(data)

,weather,base,visibility,dt,timezone,id,name,cod,coord.lon,coord.lat,...,main.sea_level,main.grnd_level,wind.speed,wind.deg,clouds.all,sys.type,sys.id,sys.country,sys.sunrise,sys.sunset
0,"[{'id': 800, 'main': 'Clear', 'description': '...",stations,10000,1731565215,32400,1835848,Seoul,200,126.9778,37.5683,...,1019,1013,1.54,90,0,1,8105,KR,1731535849,1731572545


In [33]:
df_weather = pd.json_normalize(data) 
df_weather["measured_at"] = pd.to_datetime(df_weather["dt"], unit="s") # 기준시간 
df_weather["dt"] = df_weather["measured_at"].dt.strftime('%Y%m%d') # 기준년월일 (YYYYMMDD) 
df_weather["time"] = df_weather["measured_at"].dt.strftime('%H%M%S') # 기준년월일 (HHHHMMSS) 
df_selected = df_weather[["dt", "time", "measured_at", "id", "name", "main.temp", "main.humidity", "wind.speed"]]
df_selected = df_selected.rename( # 컬럼명 수정 
    columns={"name": "city", "main.temp": "temperature", "main.humidity": "humidity", "wind.speed": "wind_speed"}
)
df_selected

,dt,time,measured_at,id,city,temperature,humidity,wind_speed
0,20241114,062015,2024-11-14 06:20:15,1835848,Seoul,17.51,59,1.54


### 04. Pandas 로 CSV 파일 저장

In [34]:
# DataFrame 생성 및 CSV 파일로 저장
df = pd.DataFrame(df_selected)
df.to_csv('weather_api.csv', index=False)

print("날씨 정보가 CSV 파일로 저장되었습니다.")

날씨 정보가 CSV 파일로 저장되었습니다.


### 05. Class로 만들어보자 

In [ ]:
import requests


class WeatherApiClient:
    """
    OpenWeatherMap API 클라이언트를 사용하여 날씨 데이터를 가져오는 클래스입니다.
    """

    def __init__(self, api_key: str):
        self.base_url = "http://api.openweathermap.org/data/2.5"
        if api_key is None:
            raise Exception("API 키는 None으로 설정할 수 없습니다.")
        self.api_key = api_key

    def get_city(self, city_name: str, temperature_units: str = "metric") -> dict:
        """
        지정된 도시의 최신 날씨 데이터를 가져옵니다.

        Parameters:
        - city_name (str): 날씨 정보를 조회할 도시 이름.
        - temperature_units (str): 온도 단위 (기본값은 'metric'으로 섭씨 기준).
                                'metric'은 섭씨, 'imperial'은 화씨, 'standard'는 켈빈 단위를 의미합니다.

        Returns:
        - dict: 요청한 도시의 날씨 데이터가 포함된 JSON 응답을 반환합니다.

        Raises:
        - Exception: API 요청이 실패한 경우 상태 코드와 응답 메시지와 함께 예외가 발생합니다.
        """
        params = {"q": city_name, "units": temperature_units, "appid": self.api_key}
        response = requests.get(f"{self.base_url}/weather", params=params)
        if response.status_code == 200:
            return response.json()
        else:
            raise Exception(
                f"Open Weather API에서 데이터를 추출하지 못했습니다. 상태 코드: {response.status_code}. 응답: {response.text}"
            )

In [ ]:
import pandas as pd 
from dotenv import load_dotenv
import os 
load_dotenv()

weather_api_client = WeatherApiClient(api_key = os.environ.get("API_KEY")) 
data = weather_api_client.get_city("Seoul") 

# 간단한 전처리
df_weather = pd.json_normalize(data) # JSON 데이터를 Pandas 형태로 변환 
df_weather["measured_at"] = pd.to_datetime(df_weather["dt"], unit="s") # 기준시간 
df_weather["dt"] = df_weather["measured_at"].dt.strftime('%Y%m%d') # 기준년월일 (YYYYMMDD) 
df_weather["time"] = df_weather["measured_at"].dt.strftime('%H%M%S') # 기준년월일 (HHHHMMSS) 
df_selected = df_weather[["dt", "time", "measured_at", "id", "name", "main.temp", "main.humidity", "wind.speed"]]
df_selected = df_selected.rename( # 컬럼명 수정 
    columns={"name": "city", "main.temp": "temperature", "main.humidity": "humidity", "wind.speed": "wind_speed"}
)

# DataFrame 생성 및 CSV 파일로 저장
df = pd.DataFrame(df_selected)
df.to_csv('weather_api.csv', index=False)